In [1]:
from tqdm.notebook import tqdm as progress
import pandas as pd
import plotly.express as px
from plotly.offline import plot
import dimcli
from dimcli.utils import *
import os, sys, time, json
from IPython.display import Image
from IPython.core.display import HTML 
import csv
import ast

/Users/FionaXu/Documents/Research/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning:

urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020



In [2]:
#Log into Dimensions
#API key is: 9F8D648F0D7E437CB1736BEBDF007F02
#!pip install dimcli -U --quiet 

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
    import getpass
    KEY = getpass.getpass(prompt='API Key: ')  
    dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
    KEY = "9F8D648F0D7E437CB1736BEBDF007F02"
    dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()



==
Logging in..
Dimcli - Dimensions API Client (v1.4)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.10
Method: manual login


In [5]:

#Link to grant variables description: https://docs.dimensions.ai/dsl/datasource-grants.html#grants
search_Var = "[abstract + active_year + category_bra + category_for + category_for_2020 + category_hra + category_hrcs_hc + category_hrcs_rac + category_icrp_cso + category_icrp_ct + category_rcdc + category_sdg + category_uoa + concepts + concepts_scores + date_inserted + dimensions_url + end_date + foa_number + funder_countries + funder_org_acronym + funder_org_cities + funder_org_countries + funder_org_name + funder_org_states + funder_orgs + funders + funding_aud + funding_cad + funding_chf + funding_cny + funding_currency + funding_eur + funding_gbp + funding_jpy + funding_nzd + funding_org_acronym + funding_org_cities + funding_org_city + funding_org_name + funding_org_states + funding_schemes + funding_usd + grant_number + id + investigators + keywords + language + language_title + linkout + original_title + project_numbers + research_org_cities + research_org_countries + research_org_names + research_org_state_codes + research_orgs + researchers + score + start_date + start_year + title]"
names = {}

#CHANGE CSV FILE TO THE NAME OF YOUR FILE SAVED FROM DimensionsAI_IDs.ipynb
with open('dimensionsAI_NameIds.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)  
    for row in reader:
        name = row[0]
        openIds_str = row[1]  
        openIds_list = ast.literal_eval(openIds_str)
        names[name] = openIds_list

print(names)


{'Sarki Abdulkadir': ['ur.01151331063.92'], 'Ash Alizadeh': ['ur.01350734042.58', 'ur.015743252105.61'], 'Vineet Arora': ['ur.0715157556.43', 'ur.013113430640.74', 'ur.011030560726.50', 'ur.014736613175.67', 'ur.014621242702.28', 'ur.07366375701.95', 'ur.014365324233.20', 'ur.014067114314.28'], 'Andrew Auerbach': ['ur.012611444224.52', 'ur.01071266221.30', 'ur.015005166353.24', 'ur.013532372135.43', 'ur.016370705613.19', 'ur.015401373657.32', 'ur.014116621270.69', 'ur.012057467657.83', 'ur.01316550603.31', 'ur.014005474317.25', 'ur.010354504201.10'], 'Adam Bass': ['ur.010517020604.25', 'ur.01012315545.30', 'ur.0776611211.51', 'ur.01000517455.50', 'ur.014426110660.10', 'ur.013227242701.97', 'ur.014713043230.45', 'ur.01111016005.45', 'ur.010534015564.00', 'ur.015401503732.91', 'ur.014662725755.25', 'ur.01351310303.21'], 'Edward Behrens': ['ur.016043042647.75', 'ur.014217023641.56', 'ur.014742005621.45', 'ur.011112113147.76', 'ur.016455011753.14', 'ur.010446365434.45', 'ur.011574150043.14

In [6]:
def findby_idGrant(nameDict):
    all_Grants= {}
    count=0
    impactCodes = ['R01', 'R03', 'R13', 'R15', 'R21', 'R34', 'R41', 'R42', 'R43', 'R44', 'R56', 'R29', 'F05', 'F30', 'F31', 'F32', 'F33', 'F37', 'F38', 'F99', 'F12', 'FM1', 'R2F', 'RF1']
    for name, person_ids in nameDict.items():
        indv_grant_list = []
        for i in person_ids:
            numGrants = 0
            res = dsl.query_iterative(f"""search grants where researchers = "{i}" return grants{search_Var}""")
            for grant in res.json['grants']: #Filtering out grants with 5 or more investigators and those that are NIH but don't have impactful codes
                caught = [code for code in impactCodes if(code in grant['grant_number'])] #bool(caught) returns false for empty list (if grant has no impactful code)
                if(len(grant['investigators']) < 5 and (bool(caught) or not "National" in grant['funder_org_name'])):
                    indv_grant_list.append(grant) 
                    numGrants+=1
            number = res.json['_stats']['total_count']
            print(f'{i} id has {number} total grants')
            print(f'{i} id has {numGrants} high impact grants')
            listDdict = res.json.get('grants', [])

            indv_grant_list.extend(listDict)

        all_Grants[name] = indv_grant_list
        count+=1
        print(f'{count}. {name}')
        print(len(all_Grants[name]))
    return all_Grants

grants = findby_idGrant(names) 

json_filename = "all_Grants.json"

with open(json_filename, 'w') as json_file:
    json.dump(grants, json_file, indent=2)

print(f'The data has been written to {json_filename}')

Starting iteration with limit=1000 skip=0 ...
0-11 / 11 (0.52s)
===
Records extracted: 11
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


ur.01151331063.92 id has 11 total grants
ur.01151331063.92 id has 9 high impact grants
1. Sarki Abdulkadir
20


0-11 / 11 (0.36s)
===
Records extracted: 11
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


ur.01350734042.58 id has 11 total grants
ur.01350734042.58 id has 9 high impact grants


===
Records extracted: 0
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


...

ur.012106750212.06 id has 0 total grants
ur.012106750212.06 id has 0 high impact grants


0-10 / 10 (0.39s)
===
Records extracted: 10
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


ur.01144174070.48 id has 10 total grants
ur.01144174070.48 id has 2 high impact grants


0-1 / 1 (4.33s)
===
Records extracted: 1
Warnings:  8


ur.07423033552.33 id has 1 total grants
ur.07423033552.33 id has 0 high impact grants
149. David Golan
15
The data has been written to all_Grants.json


In [2]:
import csv
import json

csv_filename = "dimensionsGrants_data.csv"

with open("all_Grants.json", 'r', encoding='utf-8') as file:
    personDict = json.load(file)
# removed 'investigators
common_work_info_headers = ['abstract', 'active_years', 'altmetric', 'associated_grant_ids', 'brief_title', 'category_bra', 'category_for', 'category_for_2020', 'category_hra', 'category_hrcs_rac', 'category_rcdc', 'conditions', 'date', 'date_inserted', 'dimensions_url', 'end_date', 'funder_countries', 'funders', 'funding_usd','grant_number','gender', 'id', 'interventions', 'linkout', 'mesh_terms', 'phase', 'publication_ids', 'publications', 'registry', 'score', 'start_date', 'study_arms', 'study_designs', 'study_eligibility_criteria', 'study_maximum_age', 'study_minimum_age', 'study_outcome_measures', 'study_participants', 'study_type', 'title']


with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    
    header = ['Person'] + common_work_info_headers
    writer.writerow(header)

    # Write data rows
    for person, works in personDict.items():
        for work in works:  # Assuming works is a list of dictionaries now
            row = [person] + [work.get(head, '') for head in common_work_info_headers]
            writer.writerow(row)

print(f"CSV file '{csv_filename}' created successfully.")

CSV file 'dimensionsGrants_data.csv' created successfully.
